# 第四次讀書會：LLM Information Extraction

# 連接雲端資料夾

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/') #切換該目錄
os.listdir() #確認目錄內容

Mounted at /content/drive


['requirements.txt', 'dict', 'data', 'LLM_information_extraction.ipynb']

# 套件安裝

In [ ]:
!pip install jieba

!pip install sentence-transformers
!pip install transformers
!pip install sentencepiece

# 用於 transformer 加速
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

!pip install langchain-chroma
!pip install langchain_community
!pip install pypdf
!pip install langchain
!pip install huggingface_hub
!pip install grandalf
!pip install pandas
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import os
from getpass import getpass

import pandas as pd
import ast
import json
import re
import jieba

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import BitsAndBytesConfig # huggingface 量化

from langchain_community.llms import HuggingFaceEndpoint

from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# HuggingFace Token

輸入 HuggingFace Token
- get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

In [ ]:
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
# repo_id: HuggingFace 上的模型名稱，本範例使用 gemma 7b instruct 版本
    # huggingface: https://huggingface.co/google/gemma-1.1-7b-it
# temperature: 模型回答的活潑程度，0~1 越大每次回答的多樣性越高

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-1.1-7b-it", temperature=1.0,model_kwargs={'token':HUGGINGFACEHUB_API_TOKEN, 'use_cache':False}
)
llm.invoke("你好，自然語言處理有什麼相關應用？")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


'\n\n**自然語言處理（NLP）的相關應用：**\n\n**1. 機器翻譯：**\n* 自動翻譯文字和語音，提高人際交流的效率。\n\n**2. 情緒分析：**\n* 分析文本的情緒和情緒內容。\n\n**3. 情感偵測：**\n* 建立對話系統和聊天機器人的情緒感知能力。\n\n**4. 文本簡約：**\n* 將長文簡化為更簡潔的版本。\n\n**5. 詞彙預測：**\n* 基於使用者歷史數據預測下一個詞的可能性。\n\n**6. 文本摘要：**\n* 自動產生文檔的摘要。\n\n**7. 語言轉換：**\n* 將一組語言轉換為另一組語言。\n\n**8. 自動標記：**\n* 在文件和電子郵件中自動標記重要詞彙。\n\n**9. 電子郵件過濾：**\n* 基於內容過濾電子郵件。\n\n**10. 社交媒體分析：**\n* 分析社交媒體數據以了解趨勢和個人興趣。\n\n\n**自然語言處理的未來應用：**\n\n* 語言理解和生成的進步。\n* 人機協作和輔助。\n* 基於語言的健康監測和診斷。'

In [ ]:
llm.invoke("你好，日本旅遊有什麼好玩或是好吃的地訪嗎？")

'\n\n**日本旅遊推薦：**\n\n**淺草：**以其茶葉農場著稱，還有淺草寺和記平亭等觀光景點。\n\n**銀座：**以其購物街著稱，有許多高端品牌和餐廳。\n\n**新宿：**以其高樓大廈和夜生活著稱。\n\n**京都：**以其古色古香的傳統文化著稱，有許多寺廟和古蹟。\n\n**大阪：**以其美食和購物著稱，還有日本最大的展望台。\n\n**ValueGeneration地：**以其湖泊和瀑布著稱，還有丹波湖和飛騨瀑布等觀光景點。\n\n**名古屋：**以其現代建築和美食著稱。\n\n**函館：**以其漁村和自然景觀著稱。\n\n\n**日本美食推薦：**\n\n**寿司：**以其薄切的魚肉和海果為食著稱。\n\n**tempura：**以其炸入熱油的食品著稱，通常以蝦仁或根菜等作爲材料。\n\n**tonkatsu：**以其炸入熱油的豬肉排著稱。\n\n**karaage：**以其炸入熱油的雞肉或蔬菜著稱。\n\n**okonomiyaki：**以其薄面片和各種食材混合著稱，通常以肉末或海藻等作爲材料。'

可以看到上述提問與日本旅遊的相關問題後，得到的回覆包括"淺草、銀座、新宿、京都、大阪、名古屋"等著名景點，並且有對各個景點進行簡略的說明。<br>
除此之外，也說明了日本的美食，像是"壽司、天婦羅等等"。<br>

#### RunnableSequence

In [ ]:
template = """你是一位社群媒體分析專家，請問以下句子的情緒為何（正向,中性,負向）: {question}"""

prompt = PromptTemplate.from_template(template)

prompt_output = prompt.invoke({"question": '日本東京有非常多好玩的景點!'})
prompt_output

StringPromptValue(text='你是一位社群媒體分析專家，請問以下句子的情緒為何（正向,中性,負向）: 日本東京有非常多好玩的景點!')

In [ ]:
print(prompt_output)
llm_output = llm.invoke(prompt_output)
llm_output

text='你是一位社群媒體分析專家，請問以下句子的情緒為何（正向,中性,負向）: 日本東京有非常多好玩的景點!'


'\n\n**情緒分析：**\n\n* 正向\n* 中性\n* 負向\n\n\n**說明：**\n\n句子的情緒為**正向**。這句子表達出對日本東京景點的積極觀點，並表示它們很適合遊客遊覽。'

In [ ]:
# RunnablePassthrough: 可以不變的傳遞整個 input

template = """你是一位社群媒體分析專家，請幫以下句子的評分情緒分數（介於 -1.0~1.0，越高分代表越正向，反之越負向）: {question}"""

prompt = PromptTemplate.from_template(template)

sentiment_value_chain = (
    {"question": RunnablePassthrough()}
    | prompt
    | llm
)
sentiment_value_chain.invoke("東京、京都、大阪都有非常多著名的景點")

'和美食，是日本最熱鬧的都市。\n\n**情緒分數：**0.8\n\n**評估說明：**該句子的情緒主旨為積極，因為它表達了對日本都市景點和美食的讚美，以及對日本都市熱鬧的肯定。'

#### RunnableParallel

In [ ]:
# 設定繁體中文詞庫
jieba.set_dictionary("./dict/dict.txt.big")

# 新增stopwords
# jieba.analyse.set_stop_words('./dict/stop_words.txt') #jieba.analyse.extract_tags才會作用
with open("./dict/stop_words.txt", encoding="utf-8") as f:
    stopWords = [line.strip() for line in f.readlines()]

# 設定斷詞 function
def getToken(row):
    seg_list = jieba.cut(row, cut_all=False)
    seg_list = [
        w for w in seg_list if w not in stopWords and len(w) > 1
    ]  # 篩選掉停用字與字元數大於1的詞彙
    return seg_list

# 讀取情緒字典
liwc_dict = pd.read_csv("./dict/liwc/LIWC_CH.csv")
liwc_dict = liwc_dict.rename(columns={'name': 'word', "class": 'sentiments'})



In [ ]:
# call lexicon base function
def lexicon_base_sentiment_value(value: dict) -> str:
    tokens = getToken(value['question'])
    df = pd.DataFrame(tokens, columns=['word'])
    liwc_df = pd.merge(df, liwc_dict, how="left")
    sentiment_count = pd.DataFrame(
        liwc_df.groupby(["sentiments"]).size()
    ).reset_index()
    result_dict = sentiment_count.set_index('sentiments')[0].to_dict()
    print(result_dict)
    s_value = (result_dict.get('positive') if result_dict.get('positive') != None else 0) - (result_dict.get('negative') if result_dict.get('negative') != None else 0)
    return s_value


lexicon_sentiment_value_chain = RunnableLambda(lexicon_base_sentiment_value)

parallel = {"runnable_1": lexicon_sentiment_value_chain, "runnable_2": sentiment_value_chain}

chain = RunnableLambda(lambda x: x) | parallel
answer = chain.invoke({'question':'東京很好玩，有些東西偏貴，而且有些景點觀光客太多了，但是環境整體來說都很乾淨，應該還會再回訪。'})

print(answer)

Building prefix dict from /content/drive/MyDrive/SNA/week13/dict/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /content/drive/MyDrive/SNA/week13/dict/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u3c478e9f7c1e351c29dd0395e4626710.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.u3c478e9f7c1e351c29dd0395e4626710.cache
Loading model cost 2.001 seconds.
DEBUG:jieba:Loading model cost 2.001 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


{'affect': 2, 'positive': 2}
{'runnable_1': 2, 'runnable_2': '\n\n**評分結果：0.7**\n\n**評分說明：**這句子表現出對東京的積極看法，即使有一些缺點，例如貴和人潮，也認為環境整體乾淨，並且表示會再回訪。因此，情緒分數為 0.7。'}


In [ ]:
chain = RunnableParallel(lexicon=lexicon_sentiment_value_chain, LLM=sentiment_value_chain)
chain.invoke({'question':'日本人都很友善!美食也超好吃!'})

{'affect': 2, 'positive': 2}


{'lexicon': 2,
 'LLM': '\n\n**評分：** 0.8\n\n**解釋：** 該句子表達了積極的情緒，因為它包含了對日本人民和美食的讚美。其積極的情緒表現出於：\n- 對日本人民的友善之言。\n- 對日本美食的讚美。'}

# 3. 資料集實作 Information Extraction (Tagging)
- https://python.langchain.com/docs/use_cases/tagging/
- 資料來源: Tarflow，PTT日本旅遊版

In [ ]:
df = pd.read_csv("./data/Japan_Travel.csv")
df["artDate"] = pd.to_datetime(df["artDate"])
df['artDate'] = df['artDate'].dt.date

clear_df = df.copy()

#去除本次不會用到的欄位
drop_cols = ['system_id', 'artPoster', 'artCatagory', 'artComment', 'e_ip', 'insertedDate', 'dataSource']
clear_df.drop(drop_cols, axis = 1, inplace = True)

#去除文章內容為空值的筆數
clear_df.dropna(subset = ['artContent'], axis=0, how='any', inplace=True)

#新增['sentence']欄位，用'。'取代'\n\n'，並移除'\n'
clear_df['sentence'] = clear_df['artContent'].str.replace(r'\n\n','。', regex=True)
clear_df['sentence'] = clear_df['sentence'].str.replace(r'\n','', regex=True)

#移除內文中的網址
clear_df['sentence'] = clear_df['sentence'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

clear_df['sentence'] = clear_df['sentence'].str.split("[,，。！!？?]{1,}")

sent_df = clear_df.explode('sentence').reset_index(drop=True)
# 把長度小於1的sentence刪掉
sent_df = sent_df[sent_df["sentence"].str.len() > 1]

sampled_df = sent_df.sample(n=100, random_state=222)

sampled_df

,artUrl,artTitle,artDate,artContent,sentence
101277,https://www.ptt.cc/bbs/Japan_Travel/M.16849138...,[問題]一日遊選擇：鎌倉、橫濱或其他建議,2023-05-24,各位版大大家好：\n在下安排6月底7月初再到東京進行旅遊，\n但後來才發現是日本梅雨季orz...,3. 或是東京市區推薦一日遊的地方
101545,https://www.ptt.cc/bbs/Japan_Travel/M.16849900...,[遊記]9天8夜茨城及東京近郊之旅,2023-05-25,造訪日期：2023.5.14~5.22\n2023.5.14虎航桃園機場飛茨城空港\n202...,總覺得都能出道當漫畫家了XD
34600,https://www.ptt.cc/bbs/Japan_Travel/M.16001626...,[資訊]日本自助個人心得及經驗分享,2020-09-15,大家好，第一次寫大型遊戲【玩樂日本Online】攻略XD，文長字多，\n因為我不是專業的，請...,所以他們除了口頭問話還會以一張圖文並茂的手板給你看
66953,https://www.ptt.cc/bbs/Japan_Travel/M.16708597...,[遊記]晚秋保津川，京都龜岡保津川遊船初體驗,2022-12-12,京都龜岡・嵐山保津川遊船\n\n造訪日期：2022年/11月\n地 址：京都府亀岡市保津町...,這顆石頭被他們稱為「象さん」（中文：大象先生）
92465,https://www.ptt.cc/bbs/Japan_Travel/M.16807717...,Re:[問題]大阪奈良五天四夜自駕親子遊請益,2023-04-06,今天返台，來分享一下自駕心得\n\n結論:\n\n=====================...,可能剛好因為下雨
...,...,...,...,...,...
75388,https://www.ptt.cc/bbs/Japan_Travel/M.16732389...,[遊記]雙寶2打2雙人推車東京7天6夜12/21～12/2,2023-01-09,大寶：3Y9M\n二寶：2Y1M\n小寶：肚子裡4M\n行囊：vista推車一台，20吋/2...,就這些晚上回飯店之前就是跑一次超市補貨
74604,https://www.ptt.cc/bbs/Japan_Travel/M.16730631...,[遊記]九州遊記【十五】水前寺成趣園,2023-01-07,造訪日期：2019年四月（耽擱許久才寫完，請見諒）\n\n圖文網誌版:\nhttps://b...,我盯著當中的漢字
140717,https://www.ptt.cc/bbs/Japan_Travel/M.17022786...,[問題]旭川札幌景點請益,2023-12-11,今年12月底預計去北海道 富良野滑雪\n其中有一天會從旭川搭JR回札幌\n\n想請問 從旭川...,今年12月底預計去北海道 富良野滑雪其中有一天會從旭川搭JR回札幌
34224,https://www.ptt.cc/bbs/Japan_Travel/M.16001626...,[資訊]日本自助個人心得及經驗分享,2020-09-15,大家好，第一次寫大型遊戲【玩樂日本Online】攻略XD，文長字多，\n因為我不是專業的，請...,同間飯店同間房（假設性舉例）A網站一晚1800元


In [ ]:
class Food(BaseModel):
    情緒: str = Field(..., description="判斷整個句子總體的情緒。", enum=["正向", "中性", "負向"])
    景點: str = Field(
        ...,
        description="列出所有句子中提及的地名並以逗號','分隔。"
    )
    地名種類: str = Field(
        ...,
        description="判斷句子中提到的景點是日本的哪個縣市。",
        enum=["東京", "京都", "北海道", "沖繩", "大阪", "無"],
    )

food_parser = JsonOutputParser(pydantic_object=Food)

# 透過 get_format_instructions() 可以看到如何告訴 LLM 生成我們想要的格式的一些指令
food_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"\\u60c5\\u7dd2": {"title": "\\u60c5\\u7dd2", "description": "\\u5224\\u65b7\\u6574\\u500b\\u53e5\\u5b50\\u7e3d\\u9ad4\\u7684\\u60c5\\u7dd2\\u3002", "enum": ["\\u6b63\\u5411", "\\u4e2d\\u6027", "\\u8ca0\\u5411"], "type": "string"}, "\\u666f\\u9ede": {"title": "\\u666f\\u9ede", "description": "\\u5217\\u51fa\\u6240\\u6709\\u53e5\\u5b50\\u4e2d\\u63d0\\u53ca\\u7684\\u5730\\u540d\\u4e26\\u4ee5\\u9017\\u865f\',\'\\u5206\\u9694\\u3002", "type": "string"}, "\\u5730\\u540d\\u7a2e\\u985e": {"title": "\\u5730\\u540d\\u7a2e\\u985e

In [ ]:
# Set up a parser + inject instructions into the prompt template.
food_parser = JsonOutputParser(pydantic_object=Food)\

food_template = """<start_of_turn>user
你是一位日本旅遊評論家，你將會看網路上對於日本旅遊的評論，請抓取出評論中：
1.此評論的情緒，正向或負向或中性
2.有討論到地名，例如：東京、京都、大阪
3.地名相關的景點，例如：迪士尼、晴空塔、神社

以下為一些範例：
```
範例1
句子：到東京旅遊會造訪的景點一定是晴空塔
抓取結果：{{
    "情緒":"中性",
    "景點":"晴空塔",
    "地名種類":"東京"
}}

範例2
句子：實際欣賞大阪城果然非常壯觀!
抓取結果：{{
    "情緒":"正向",
    "景點":"大阪城",
    "地名種類":"無"
}}

範例3
句子：東京的食物和景點真的好吃又好玩，可惜沒有去迪士尼逛逛
抓取結果：{{
    "情緒":"正向",
    "景點":"迪士尼",
    "地名種類":"東京"
}}

範例4
句子：其實也還好
抓取結果：{{
    "情緒":"中性",
    "景點":"無",
    "地名種類":"無"
}}


{format_instructions}
請使用繁體中文回答。
除了 JSON 內容外，不要回答任何其他內容

現在請幫我抓取以下句子的資訊：
句子：{query}
抓取結果：<end_of_turn>
<start_of_turn>model
"""

food_prompt = PromptTemplate(
    template=food_template,
    # template="Answer the user query.\n{format_instructions}\nDon't output any other explanation\n{query}\n",
    input_variables=["query"],
    # 將 format_instructions 也輸入到 prompt templete 中
    partial_variables={"format_instructions": food_parser.get_format_instructions()},
)

chain = food_prompt | llm
chain.get_graph().print_ascii()

       +-------------+         
       | PromptInput |         
       +-------------+         
              *                
              *                
              *                
      +----------------+       
      | PromptTemplate |       
      +----------------+       
              *                
              *                
              *                
   +---------------------+     
   | HuggingFaceEndpoint |     
   +---------------------+     
              *                
              *                
              *                
+---------------------------+  
| HuggingFaceEndpointOutput |  
+---------------------------+  


手動建立 parse function

In [ ]:
# 從 json 字串中抓取答案
def extract_answer_from_string(string_):
    string_ = string_.replace("\n", "")
    string_ = string_.replace("json", "")
    string_ = string_.replace("    ", "")
    string_ = string_.replace("```", "")
    try:
        # 可以成功轉換
        dict_result = json.loads(string_)
    except:
        print(string_)
        return string_
    return dict_result

# 從 dict 抓取情緒
def get_sentiment(row):
    sentiment = None
    if isinstance(row, dict):
        if ("情緒" in row.keys()):
            sentiment = row['情緒']
    return sentiment

In [ ]:
llm_result = extract_answer_from_string(chain.invoke({"query": "晴空塔真的有夠高，而且頂樓觀景台看到的景色很漂亮"}))
print(llm_result)
get_sentiment(llm_result)

{'情緒': '正向', '景點': '晴空塔', '地名種類': '東京'}


'正向'

上述得到的分析結果，有清楚的將"晴空塔"視為一個景點，並且清楚的說明其所在的城市為"東京"。

In [ ]:
sampled_df['LLM'] = sampled_df.apply(lambda x: extract_answer_from_string(chain.invoke({"query": x['sentence']})), axis=1)
sampled_df

The provided text does not contain any sentences regarding traveling to Japan, so I am unable to extract the requested information.
資訊不足，無法抓取所需資訊。
The provided sentence does not include information regarding the date of visit, so the requested data could not be extracted.


,artUrl,artTitle,artDate,artContent,sentence,LLM
101277,https://www.ptt.cc/bbs/Japan_Travel/M.16849138...,[問題]一日遊選擇：鎌倉、橫濱或其他建議,2023-05-24,各位版大大家好：\n在下安排6月底7月初再到東京進行旅遊，\n但後來才發現是日本梅雨季orz...,3. 或是東京市區推薦一日遊的地方,"{'情緒': '中性', '景點': '東京市區推薦一日遊的地方', '地名種類': '東京'}"
101545,https://www.ptt.cc/bbs/Japan_Travel/M.16849900...,[遊記]9天8夜茨城及東京近郊之旅,2023-05-25,造訪日期：2023.5.14~5.22\n2023.5.14虎航桃園機場飛茨城空港\n202...,總覺得都能出道當漫畫家了XD,"{'情緒': '正向', '景點': '無', '地名種類': '無'}"
34600,https://www.ptt.cc/bbs/Japan_Travel/M.16001626...,[資訊]日本自助個人心得及經驗分享,2020-09-15,大家好，第一次寫大型遊戲【玩樂日本Online】攻略XD，文長字多，\n因為我不是專業的，請...,所以他們除了口頭問話還會以一張圖文並茂的手板給你看,"{'情緒': '正向', '景點': '無', '地名種類': '無'}"
66953,https://www.ptt.cc/bbs/Japan_Travel/M.16708597...,[遊記]晚秋保津川，京都龜岡保津川遊船初體驗,2022-12-12,京都龜岡・嵐山保津川遊船\n\n造訪日期：2022年/11月\n地 址：京都府亀岡市保津町...,這顆石頭被他們稱為「象さん」（中文：大象先生）,"{'情緒': '中性', '景點': '無', '地名種類': '無'}"
92465,https://www.ptt.cc/bbs/Japan_Travel/M.16807717...,Re:[問題]大阪奈良五天四夜自駕親子遊請益,2023-04-06,今天返台，來分享一下自駕心得\n\n結論:\n\n=====================...,可能剛好因為下雨,"{'情緒': '中性', '景點': '無', '地名種類': '無'}"
...,...,...,...,...,...,...
75388,https://www.ptt.cc/bbs/Japan_Travel/M.16732389...,[遊記]雙寶2打2雙人推車東京7天6夜12/21～12/2,2023-01-09,大寶：3Y9M\n二寶：2Y1M\n小寶：肚子裡4M\n行囊：vista推車一台，20吋/2...,就這些晚上回飯店之前就是跑一次超市補貨,"{'情緒': '中性', '景點': '超市', '地名種類': '無'}"
74604,https://www.ptt.cc/bbs/Japan_Travel/M.16730631...,[遊記]九州遊記【十五】水前寺成趣園,2023-01-07,造訪日期：2019年四月（耽擱許久才寫完，請見諒）\n\n圖文網誌版:\nhttps://b...,我盯著當中的漢字,"{'情緒': '中性', '景點': '無', '地名種類': '無'}"
140717,https://www.ptt.cc/bbs/Japan_Travel/M.17022786...,[問題]旭川札幌景點請益,2023-12-11,今年12月底預計去北海道 富良野滑雪\n其中有一天會從旭川搭JR回札幌\n\n想請問 從旭川...,今年12月底預計去北海道 富良野滑雪其中有一天會從旭川搭JR回札幌,"{'情緒': '正向', '景點': 'JR', '地名種類': '北海道'}"
34224,https://www.ptt.cc/bbs/Japan_Travel/M.16001626...,[資訊]日本自助個人心得及經驗分享,2020-09-15,大家好，第一次寫大型遊戲【玩樂日本Online】攻略XD，文長字多，\n因為我不是專業的，請...,同間飯店同間房（假設性舉例）A網站一晚1800元,"{'情緒': '中性', '景點': '無', '地名種類': '無', '價格': '18..."


In [ ]:
sampled_df['sentiment'] = sampled_df.apply(lambda x: get_sentiment(x['LLM']), axis=1)
sampled_df.head()

,artUrl,artTitle,artDate,artContent,sentence,LLM,sentiment
101277,https://www.ptt.cc/bbs/Japan_Travel/M.16849138...,[問題]一日遊選擇：鎌倉、橫濱或其他建議,2023-05-24,各位版大大家好：\n在下安排6月底7月初再到東京進行旅遊，\n但後來才發現是日本梅雨季orz...,3. 或是東京市區推薦一日遊的地方,"{'情緒': '中性', '景點': '東京市區推薦一日遊的地方', '地名種類': '東京'}",中性
101545,https://www.ptt.cc/bbs/Japan_Travel/M.16849900...,[遊記]9天8夜茨城及東京近郊之旅,2023-05-25,造訪日期：2023.5.14~5.22\n2023.5.14虎航桃園機場飛茨城空港\n202...,總覺得都能出道當漫畫家了XD,"{'情緒': '正向', '景點': '無', '地名種類': '無'}",正向
34600,https://www.ptt.cc/bbs/Japan_Travel/M.16001626...,[資訊]日本自助個人心得及經驗分享,2020-09-15,大家好，第一次寫大型遊戲【玩樂日本Online】攻略XD，文長字多，\n因為我不是專業的，請...,所以他們除了口頭問話還會以一張圖文並茂的手板給你看,"{'情緒': '正向', '景點': '無', '地名種類': '無'}",正向
66953,https://www.ptt.cc/bbs/Japan_Travel/M.16708597...,[遊記]晚秋保津川，京都龜岡保津川遊船初體驗,2022-12-12,京都龜岡・嵐山保津川遊船\n\n造訪日期：2022年/11月\n地 址：京都府亀岡市保津町...,這顆石頭被他們稱為「象さん」（中文：大象先生）,"{'情緒': '中性', '景點': '無', '地名種類': '無'}",中性
92465,https://www.ptt.cc/bbs/Japan_Travel/M.16807717...,Re:[問題]大阪奈良五天四夜自駕親子遊請益,2023-04-06,今天返台，來分享一下自駕心得\n\n結論:\n\n=====================...,可能剛好因為下雨,"{'情緒': '中性', '景點': '無', '地名種類': '無'}",中性


In [ ]:
sampled_df['sentiment'].value_counts()

sentiment
中性    58
正向    29
負向     8
無      1
肯定     1
Name: count, dtype: int64